In [1]:
import requests

In [2]:
from flask import json,jsonify, json_available

In [103]:
def grofers(product,cityInfo):
    payload = {
        'lat' : cityInfo['lat'],
        'lon' : cityInfo['lon'],
        'q' : "Tata Sampann " + product,
        'start' : 0,
        'size' : 48
    }
    headers = {
        'auth_key': "6d7aa42b20b9e9cd73b0d58d2b91eabb19a960b8ec24d26f51f8360d3aeb9931",
        'referer': "https://grofers.com/s/?q=tata+sampann+rice&suggestion_type=0&t=1",
        'Accept-Encoding': "gzip",
        'Accept-language': "en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7",
        'app_client': "consumer_web",
        'Content-Type': "application/json",
        'Cache-Control': "no-cache",
    }
    r = requests.get("https://grofers.com/v5/search/merchants/26015/products/", headers = headers, params = payload)
    r = r.json()
    print(r.keys())
    response = {}
    response['category'] = 'Grofers'
    items = []
    for product in r['products']:
        item = {}
        url = product['name'].replace("\\","").replace("/","").replace(" ","_").lower()
        item['name'] = product['name']
        item['cost'] = product['price']
        item['unit'] = product['unit']
        item['image_url'] = product['image_url']
        item['purchase_link'] = "https://grofers.com/prn/" + url + "/prid/" + str(product['default_product_id'])
        items.append(item)
    response['items'] = items
    return response        

In [76]:
def getAddressDetails(pincode):
    if(str(pincode) in cityInfo):
        return cityInfo[str(pincode)]
    else:
        return None

In [38]:
import pandas as pd

In [42]:
df = open("/Users/yuvraj/Downloads/IN/IN.txt").read().split("\n")


In [74]:
cityInfo= {}
for d in df:
    if(len(d) > 0):
        d = d.split("\t")
        cityInfo[d[1]] = {}
        cityInfo[d[1]]['city'] = d[5]
        cityInfo[d[1]]['state'] = d[3]
        cityInfo[d[1]]['area'] = d[2]
        cityInfo[d[1]]['lat'] = d[-3]
        cityInfo[d[1]]['lon'] = d[-2]

In [122]:
def bigBasket(product):
    slug = "tata sampann " + product
    payload = {
        "slug" : slug,
        "type" : "deck"
    }
    r = requests.get("https://www.bigbasket.com/custompage/getsearchdata/?slug=tata%20sampann%20masala&type=deck")
    r = r.json()
    
    response = {}
    response['category'] = "Big Basket"
    items = []
    products = r['json_data']['tab_info'][0]['product_info']['products']
    for product in products:
        item = {}
        item['name'] = product['pc_n'] + product['p_desc']
        item['cost'] = product['sp']
        item['unit'] = product['w']
        item['purchase_url'] = 'https://www.bigbasket.com/pd/' + str(product['sku'])
        item['image_url'] = product['p_img_url']
        items.append(item)
    response['items'] = items
    return response

In [249]:
import sqlite3

In [276]:
conn = sqlite3.connect('ideathon_mma.db')
conn.execute("drop table if exists stores")
conn.execute("drop table if exists products")
conn.execute("drop table if exists store_product_map")

In [277]:
conn.execute("create table stores (id int,name text,address text,city text,state text,pin int)")

In [278]:
stores = open("/Users/yuvraj/Development/Hackathons/ideathon_mma/product_comparison/stores.csv").read().rstrip().split("\n")

In [279]:
print(stores)

['1,Big Bazaar,Meenakshi Mall Hulimavu,Bangalore,Karnataka,560076', '2,Hyper City,Vega City Mall JP Nagar,Bangalore,Karnataka,560076', '3,MK Ahmed Super Market,JP Nagar 3rd Phase,Bangalore, Karnataka,560076', '4,DreamMart,Arekere MICO Layout Stage 2,Bangalore,Karnataka,560076', '5,Ramaiyah Provisions,BTM Layout Stage 2,Bangalore,Karnataka,560076']


In [280]:
stores = [tuple(store.split(",")) for store in stores]

In [281]:
conn.execute("create table products (id int,name text,price int ,unit text,description text)")
products = open("/Users/yuvraj/Development/Hackathons/ideathon_mma/product_comparison/products.csv").read().rstrip().split("\n")
products = [tuple(product.split(",")) for product in products]


In [282]:
conn.execute("create table store_product_map (id int,store_id int, product_id int)")
inv = open("/Users/yuvraj/Development/Hackathons/ideathon_mma/product_comparison/store_inventory.csv").read().rstrip().split("\n")
inv = [tuple(inventory.split(",")) for inventory in inv]

In [283]:
conn.executemany("insert into stores values (?,?,?,?,?,?)",stores)
conn.executemany("insert into products values (?,?,?,?,?)",products)
conn.executemany("insert into store_product_map values (?,?,?)",inv)

In [284]:
conn.commit()

In [285]:
query = "SELECT * FROM store_product_map map join stores s, products p "
query = query + " on s.id = map.store_id and p.id = map.product_id"
query = query + " where p.name like ? limit 5"
items = []
param = ("%Toor%",)
for row in conn.execute(query,param):
        item = {}
        item['name'] = row[10]
        item['price'] = row[11]
        item['store_name'] = row[4]
        item['store_address'] = " ".join(row[5:8])
        items.append(item)
print(items)

[{'name': 'Tata Sampann Organic Toor Daal', 'price': 95, 'store_name': 'Big Bazaar', 'store_address': 'Meenakshi Mall Hulimavu Bangalore Karnataka'}, {'name': 'Tata Sampann Organic Toor Daal', 'price': 95, 'store_name': 'Hyper City', 'store_address': 'Vega City Mall JP Nagar Bangalore Karnataka'}, {'name': 'Tata Sampann Organic Toor Daal', 'price': 95, 'store_name': 'MK Ahmed Super Market', 'store_address': 'JP Nagar 3rd Phase Bangalore  Karnataka'}, {'name': 'Tata Sampann Organic Toor Daal', 'price': 95, 'store_name': 'DreamMart', 'store_address': 'Arekere MICO Layout Stage 2 Bangalore Karnataka'}, {'name': 'Tata Sampann Organic Toor Daal', 'price': 95, 'store_name': 'Ramaiyah Provisions', 'store_address': 'BTM Layout Stage 2 Bangalore Karnataka'}]


In [289]:
def getProducts(search_val):
    query = "SELECT * FROM store_product_map map join stores s, products p "
    query = query + " on s.id = map.store_id and p.id = map.product_id"
    query = query + " where p.name like ? limit 5"
    param = ("%" + search_val + "%",)
    items = []
    for row in conn.execute(query,param):
        item = {}
        item['name'] = row[10]
        item['price'] = row[11]
        item['store_name'] = row[4]
        item['store_address'] = " ".join(row[5:8])
        items.append(item)
    return items

In [247]:
conn.close()

In [290]:
getProducts("toor daal")

[{'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'Big Bazaar',
  'store_address': 'Meenakshi Mall Hulimavu Bangalore Karnataka'},
 {'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'Hyper City',
  'store_address': 'Vega City Mall JP Nagar Bangalore Karnataka'},
 {'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'MK Ahmed Super Market',
  'store_address': 'JP Nagar 3rd Phase Bangalore  Karnataka'},
 {'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'DreamMart',
  'store_address': 'Arekere MICO Layout Stage 2 Bangalore Karnataka'},
 {'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'Ramaiyah Provisions',
  'store_address': 'BTM Layout Stage 2 Bangalore Karnataka'}]

In [293]:
grofers("daal",cityInfo['560076'])

dict_keys(['merchant', 'meta', 'products', 'widget_response', 'widgets', 'change_store_uri', 'categories'])


{'category': 'Grofers',
 'items': [{'name': 'Tata Sampann Unpolished Arhar Dal/Toor Dal',
   'cost': 90,
   'unit': '1 kg',
   'image_url': 'https://cdn.grofers.com/app/images/products/normal/pro_39493.jpg',
   'purchase_link': 'https://grofers.com/prn/tata_sampann_unpolished_arhar_daltoor_dal/prid/39493'},
  {'name': 'Tata Sampann Besan',
   'cost': 37,
   'unit': '500 gm',
   'image_url': 'https://cdn.grofers.com/app/images/products/normal/pro_15486.jpg',
   'purchase_link': 'https://grofers.com/prn/tata_sampann_besan/prid/15486'},
  {'name': 'Tata Sampann Unpolished Yellow Moong Dal',
   'cost': 105,
   'unit': '1 kg',
   'image_url': 'https://cdn.grofers.com/app/images/products/normal/pro_23742.jpg',
   'purchase_link': 'https://grofers.com/prn/tata_sampann_unpolished_yellow_moong_dal/prid/23742'},
  {'name': 'Tata Sampann Chana Dal',
   'cost': 71,
   'unit': '1 kg',
   'image_url': 'https://cdn.grofers.com/app/images/products/normal/pro_299087.jpg',
   'purchase_link': 'https://g

In [294]:
bigBasket("daal")

{'category': 'Big Basket',
 'items': [{'name': 'Blended MasalasMasala - Paneer',
   'cost': '33.60',
   'unit': '45 gm',
   'purchase_url': 'https://www.bigbasket.com/pd/40095115',
   'image_url': 'https://www.bigbasket.com/media/uploads/p/s/40095115_1-tata-sampann-masala-paneer.jpg'},
  {'name': 'Powdered SpicesMasala - Dal Tadka',
   'cost': '48.00',
   'unit': '100 gm',
   'purchase_url': 'https://www.bigbasket.com/pd/40070775',
   'image_url': 'https://www.bigbasket.com/media/uploads/p/s/40070775_2-tata-sampann-masala-dal-tadka.jpg'},
  {'name': 'Blended MasalasMasala - Pav Bhaji',
   'cost': '57.60',
   'unit': '100 gm',
   'purchase_url': 'https://www.bigbasket.com/pd/40095119',
   'image_url': 'https://www.bigbasket.com/media/uploads/p/s/40095119_1-tata-sampann-masala-pav-bhaji.jpg'},
  {'name': 'Powdered SpicesGaram Masala',
   'cost': '65.60',
   'unit': '100 gm',
   'purchase_url': 'https://www.bigbasket.com/pd/40070774',
   'image_url': 'https://www.bigbasket.com/media/uploa

In [295]:
getProducts("daal")

[{'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'Big Bazaar',
  'store_address': 'Meenakshi Mall Hulimavu Bangalore Karnataka'},
 {'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'Hyper City',
  'store_address': 'Vega City Mall JP Nagar Bangalore Karnataka'},
 {'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'MK Ahmed Super Market',
  'store_address': 'JP Nagar 3rd Phase Bangalore  Karnataka'},
 {'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'DreamMart',
  'store_address': 'Arekere MICO Layout Stage 2 Bangalore Karnataka'},
 {'name': 'Tata Sampann Organic Toor Daal',
  'price': 95,
  'store_name': 'Ramaiyah Provisions',
  'store_address': 'BTM Layout Stage 2 Bangalore Karnataka'}]

In [298]:
def getAllProducts(search_term,pincode):
    city = cityInfo[str(pincode)]
    resp = []
    resp.append(grofers(search_term,city))
    resp.append(bigBasket(search_term))
    resp.append(getProducts(search_term))
    return resp

In [300]:
from flask import Flask
from flask import request

app = Flask(__name__)

@app.route('/comparison')
def product_comparsion():
    search_term = request.args.get('q')
    return your_function(search_term)

if __name__ == '__main__':
    app.run(debug=True)

UnsupportedOperation: not writable

In [303]:
conn = sqlite3.connect('ideathon_mma.db')
query = "SELECT * FROM products where name like ? limit 1"
param = ("%" + "Toor" + "%",)
item = {}
for row in conn.execute(query,param):
    print(row)

(1, 'Tata Sampann Organic Toor Daal', 95, '1 kg', "Toor dal is a staple in the Indian diet. It's nutritious and can be easily digested. Tata Sampann's Toor Dal has a rich flavour and aroma and provides essential amino acids for complete protein. Used in a variety of soups salads sweets and savouries. Toor dal is an essential in every household.High Protein: Dals or Pulses are known to be one of the best sources of protein. Tata Sampann Dals are unpolished and do not undergo artificial polishing with water oil or leather thereby retaining their goodness and protein content.")
